<center><img src="https://raw.githubusercontent.com/tquangsdh20/RBAC-NIST/main/.github/rbac_logo.svg"></center>
<center><img src="https://img.shields.io/github/license/tquangsdh20/rbac-nist?style=plastic"> <img src="https://img.shields.io/badge/author-Tran%20Quang%20%7C%20Nguyen%20Dinh%20Hoang%20Quy%20%7C%20Than%20Hai%20Nhat%20Minh-blue?style=plastic"> <a href="https://hcmuteduvn-my.sharepoint.com/:p:/g/personal/tquang_sdh20_hcmut_edu_vn/EZmcIMj8739LrdtF9CRbHwoBQ3NGoQUoaDMCjeEF6iZ_iA?e=dzX99T"><img src="https://img.shields.io/badge/PowerPoint-RBAC-red?style=plastic&logo=microsoftpowerpoint&logoColor=orange"> <img src="https://img.shields.io/badge/Python-3.7%7C3.8%7C3.9-blue?style=plastic&logo=python"></a></center>

## Description

- **About**: Demo for the RBAC following NIST Model - Database Security Course Work 
- **School**: HCM City University of Technology 
- **Lucturer** : [Dang Tran Khanh](http://www.cse.hcmut.edu.vn/~khanh/)

## Introduction to RBAC

#### RBAC stand for *Role Based Access Control* model which satisfied the following conditions:
- Access is determined by roles
- A user’s roles are assigned by security administrators
- A role’s permissions are assigned by security administrators

#### For instance:

<img src="https://raw.githubusercontent.com/tquangsdh20/RBAC-NIST/main/.github/rbac_example.svg">

## RBAC Models

#### There are 4 levels of RBAC Model as the followings:

- **RBAC0**: *contains the minimum functionality for an RBAC system*
- **RBAC1**: *includes the RBAC0 functionality and adds role hierarchies, which enable one role to inherit permissions from another role*
- **RBAC2**: *includes RBAC0 and adds constraints, which restrict the ways in which the components of a RBAC system may be configured*
- **RBAC3**: *contains the functionality of RBAC0, RBAC1, and RBAC2*

## The NIST Model for RBAC

### What is the NIST?

<a href="https://vi.wikipedia.org/wiki/Vi%E1%BB%87n_Ti%C3%AAu_chu%E1%BA%A9n_v%C3%A0_K%E1%BB%B9_thu%E1%BA%ADt_qu%E1%BB%91c_gia_(Hoa_K%E1%BB%B3)"><p align="center"><img src="https://raw.githubusercontent.com/tquangsdh20/RBAC-NIST/main/.github/nist.svg"></p></a>

### The RBAC following NIST Model

<a href="https://vi.wikipedia.org/wiki/Vi%E1%BB%87n_Ti%C3%AAu_chu%E1%BA%A9n_v%C3%A0_K%E1%BB%B9_thu%E1%BA%ADt_qu%E1%BB%91c_gia_(Hoa_K%E1%BB%B3)"><p align="center"><img src="https://raw.githubusercontent.com/tquangsdh20/RBAC-NIST/main/.github/rbac-nist.svg"></p></a>

**Please refer the PowerPoint file for more details about RBAC on the top**

## Implementation

### Generate the policy file for the RBAC Model

In [ ]:
from dbms import *
import json
db = DB('./model/Database_RBAC.db')
db.init_database()
db.policy_update()

### All default users - No users have permission

In [ ]:
db.get_users()
db.reset_roles_for_user('one.sdh20')

### User Assignment

In [ ]:
# Test case 0 - Assign a new role
db.UA('one.sdh20','MATH','STUDENT')
# Test case 1 - Assign one more ROLE from other SESSION
# db.UA('one.sdh20','IT','LECTURER')

# Test case 2 - Assign the same session with comflict ROLE
# try: 
#     db.UA('one.sdh20','IT','STUDENT')
# except InValidValue as e:
#     print(e)

In [ ]:
db.check_roles('one.sdh20')
db.policy_update()

### Connect to RBAC model

In [ ]:
db.policy_update()
model = OpenModel(RBAC_NIST)
model.init()

In [ ]:
model.login('one.sdh20','password',db)

### Valid Operations

In [ ]:
model.request('MATH','math.db','VIEW NAME')

In [ ]:
model.request('IT','it.db','VIEW NAME')

In [ ]:
model.request('IT','it.db','VIEW GRADE')

In [ ]:
model.request('IT','it.db','EDIT GRADE')

In [ ]:
model.login('admin','password',db)

In [ ]:
model.request('IT','it.db','EDIT INFO')

### No permission

In [ ]:
model.login('one.sdh20','password',db)

In [ ]:
model.request('MATH','math.db','VIEW GRADE')

In [ ]:
model.request('MATH','math.db','EDIT GRADE')

In [ ]:
model.request('CHEMISTRY','chemistry.db','VIEW NAME')

In [ ]:
quit()

<center><a href="https://github.com/tquangsdh20/rbac-nist"><img src="https://img.shields.io/badge/Github-RBAC_NIST-orange?style=social&logo=github"></a></center>